In [ ]:
#Code to mount Google Drive at Colab Notebook instance
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"

In [ ]:
# Huggingface libraries to run LLM.
!pip install -q -U transformers
!pip install -q -U accelerate
!pip install -q -U bitsandbytes
!pip install -q -U huggingface_hub

#LangChain related libraries
!pip install -q -U langchain

#Open-source pure-python PDF library capable of splitting, merging, cropping,
#and transforming the pages of PDF files
!pip install -q -U pypdf

#Python framework for state-of-the-art sentence, text and image embeddings.
!pip install -q -U sentence-transformers

# Vector Databses specific Libraries
!pip install -q -U faiss-gpu
!pip install -q -U chromadb

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 3.1 MB/s eta 0:00:00


In [ ]:
#from typing import List
import transformers
from transformers import AutoModelForCausalLM, AutoTokenizer, AutoConfig, BitsAndBytesConfig
import torch
from langchain.llms import HuggingFacePipeline

from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.llms import HuggingFaceHub
from langchain.vectorstores import Chroma
from langchain.chains import ConversationalRetrievalChain


device = 'cuda' if torch.cuda.is_available() else 'cpu'

print("Device:", device)
if device == 'cuda':
    print(torch.cuda.get_device_name(0))


Device: cuda
Tesla T4



# **We are using Mistral 7 Billion parameter LLM (Open source from HuggingFace)**

1. BitsAndBytesConfig:

  - load_in_4bit
  - bnb_4bit_use_double_quant
  - bnb_4bit_quant_type
  - bnb_4bit_compute_dtype

2. AutoModelForCausalLM.from_pretrained

3. AutoTokenizer.from_pretrained








In [ ]:
origin_model_path = "mistralai/Mistral-7B-Instruct-v0.1"
model_path = "filipealmeida/Mistral-7B-Instruct-v0.1-sharded"
bnb_config = BitsAndBytesConfig \
              (
                load_in_4bit=True,
                bnb_4bit_use_double_quant=True,
                bnb_4bit_quant_type="nf4",
                bnb_4bit_compute_dtype=torch.bfloat16,
              )
model = AutoModelForCausalLM.from_pretrained (model_path, trust_remote_code=True,
                                              quantization_config=bnb_config,
                                              device_map="auto")
tokenizer = AutoTokenizer.from_pretrained(origin_model_path)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

pytorch_model.bin.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

pytorch_model-00001-of-00008.bin:   0%|          | 0.00/1.89G [00:00<?, ?B/s]

pytorch_model-00002-of-00008.bin:   0%|          | 0.00/1.95G [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.47k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/72.0 [00:00<?, ?B/s]

# **Creating pipelines to run LLM at Colab notebook**

It uses

1. Huggingface transformers pipeline
  - This is an object that abstract most of the complex code from the library, offering a simple API dedicated to several tasks, including Named Entity Recognition, Masked Language Modeling, Sentiment Analysis, Feature Extraction and Question Answering.

  - We are using "text-generation" task. It's sort of chatGPT where you ask question and model respond with answer of that question based on it's intelligence.


2. LangChain HuggingFacePipeline
  - Integrating "transformers.pipeline" with LangChain.


In [ ]:
text_generation_pipeline = transformers.pipeline(
    model=model,
    tokenizer=tokenizer,
    task="text-generation",
    eos_token_id=tokenizer.eos_token_id,
    pad_token_id=tokenizer.eos_token_id,
    repetition_penalty=1.1,
    return_full_text=False,
    max_new_tokens=300,
    temperature = 0.3,
    do_sample=True,
)
mistral_llm = HuggingFacePipeline(pipeline=text_generation_pipeline)

# **Using CSV File with LangChain CSVLoader**

In [ ]:
from langchain_community.document_loaders.csv_loader import CSVLoader
# Load the CSV file
loader = CSVLoader(file_path='NIRF-2022.csv')
data = loader.load()

# Split the documents into smaller chunks
#separator=""
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
chunked_docs  = text_splitter.split_documents(data)

# Using HuggingFace embeddings
embeddings = HuggingFaceEmbeddings()

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
from langchain.vectorstores import FAISS
db = FAISS.from_documents(chunked_docs,
                          HuggingFaceEmbeddings(model_name='sentence-transformers/all-mpnet-base-v2'))


# Connect query to FAISS index using a retriever
retriever = db.as_retriever(
    search_type="similarity",
    search_kwargs={'k': 4}
)

In [ ]:
# Create the Conversational Retrieval Chain
qa_chain = ConversationalRetrievalChain.from_llm(mistral_llm, retriever,return_source_documents=True)

In [ ]:
#We will run an infinite loop to ask questions to LLM and retrieve answers untill the user wants to quit
import sys
chat_history = []
query = input('Prompt: ')
result = qa_chain({'question': query, 'chat_history': chat_history})
print('Answer: ' + result['answer'] + '\n')
chat_history.append((query, result['answer']))

Prompt: What is the rank and city of Vellore Institute of Technology ?


/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


Answer:  The rank of Vellore Institute of Technology is 12 and it is located in Vellore.



#**Using CSV file with Pandas dataframe**

In [ ]:
!pip install -q -U pandas

In [ ]:
import pandas as pd
# Load the CSV file
df = pd.read_csv('NIRF-2022.csv')

In [ ]:
from langchain.schema.document import Document
text = ""
for ind in df.index:
    text += f"{df['NAME OF THE INSTITUTE'][ind]}  {df['CITY'][ind]}  {df['SCORE'][ind]}  {df['RANK'][ind]}\n#####\n"

#Converting text to LangChain documents so that StuffDocumentsChain can understand Input
documents = Document(page_content=text, metadata={"source": "local"})

In [ ]:
print(text)

nan  nan  nan  nan
#####
Indian Institute of Technology Madras  Chennai  90.04  1.0
#####
Indian Institute of Technology, Delhi  New Delhi  88.12  2.0
#####
Indian Institute of Technology, Bombay  Mumbai  83.96  3.0
#####
Indian Institute of Technology Kanpur  Kanpur  82.56  4.0
#####
Indian Institute of Technology, Kharagpur  Kharagpur  78.89  5.0
#####
Indian Institute of Technology, Roorkee  Roorkee  76.7  6.0
#####
Indian Institute of Technology Guwahati  Guwahati  72.98  7.0
#####
National Institute of Technology, Tiruchirappalli  Tiruchirappalli  69.17  8.0
#####
Indian Institute of Technology Hyderabad  Hyderabad  68.03  9.0
#####
National Institute of Technology Karnataka, Surathkal  Surathkal  66.04  10.0
#####
Jadavpur University  Kolkata  65.68  11.0
#####
Vellore Institute of Technology  Vellore  65.53  12.0
#####
Indian Institute of Technology (Banaras Hindu University) Varanasi  Varanasi  63.51  13.0
#####
Indian Institute of Technology (Indian School of Mines)  Dhanbad  

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
# Split the documents into smaller chunks
#separators=["\n\n", "\n", " ", ""]
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=0, separators="[#####]")
chunked_docs  = text_splitter.split_documents([documents])

# Using HuggingFace embeddings
embeddings = HuggingFaceEmbeddings()

In [ ]:
from langchain.vectorstores import FAISS
db = FAISS.from_documents(chunked_docs,
                          HuggingFaceEmbeddings(model_name='sentence-transformers/all-mpnet-base-v2'))


# Connect query to FAISS index using a retriever
retriever = db.as_retriever(
    search_type="similarity",
    search_kwargs={'k': 4}
)

In [ ]:
# Create the Conversational Retrieval Chain
qa_chain = ConversationalRetrievalChain.from_llm(mistral_llm, retriever,return_source_documents=True)

In [ ]:
#We will run an infinite loop to ask questions to LLM and retrieve answers untill the user wants to quit
import sys
chat_history = []
query = input('Prompt: ')
result = qa_chain({'question': query, 'chat_history': chat_history})
print('Answer: ' + result['answer'] + '\n')
chat_history.append((query, result['answer']))

Prompt: What is the rank of Vellore Institute of Technology ?
Answer:  The rank of Vellore Institute of Technology is 12.



#WORLD RANKING COMPARISON

In [ ]:
import pandas as pd
# Load the CSV file
df = pd.read_csv('world-ranking2024.csv')

In [ ]:
from langchain.schema.document import Document
text = ""
for ind in df.index:
    text += f"{df['rank'][ind]}  {df['name'][ind]}  {df['scores_overall'][ind]}  {df['location'][ind]}\n#####\n"

#Converting text to LangChain documents so that StuffDocumentsChain can understand Input
documents = Document(page_content=text, metadata={"source": "local"})

In [ ]:
print(text)

1  University of Oxford  98.5  United Kingdom
#####
2  Stanford University  98  United States
#####
3  Massachusetts Institute of Technology  97.9  United States
#####
4  Harvard University  97.8  United States
#####
5  University of Cambridge  97.5  United Kingdom
#####
6  Princeton University  96.9  United States
#####
7  California Institute of Technology  96.5  United States
#####
8  Imperial College London  95.1  United Kingdom
#####
9  University of California, Berkeley  94.6  United States
#####
10  Yale University  94.2  United States
#####
11  ETH Zurich  93.1  Switzerland
#####
12  Tsinghua University  92.4  China
#####
13  The University of Chicago  92.1  United States
#####
14  Peking University  91.8  China
#####
15  Johns Hopkins University  91.1  United States
#####
16  University of Pennsylvania  91  United States
#####
17  Columbia University  90.9  United States
#####
18  University of California, Los Angeles  90.1  United States
#####
19  National University of Singa

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
# Split the documents into smaller chunks
#separators=["\n\n", "\n", " ", ""]
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=0, separators="[#####]")
chunked_docs  = text_splitter.split_documents([documents])

# Using HuggingFace embeddings
embeddings = HuggingFaceEmbeddings()

In [ ]:
from langchain.vectorstores import FAISS
db = FAISS.from_documents(chunked_docs,
                          HuggingFaceEmbeddings(model_name='sentence-transformers/all-mpnet-base-v2'))


# Connect query to FAISS index using a retriever
retriever = db.as_retriever(
    search_type="similarity",
    search_kwargs={'k': 4}
)

In [ ]:
# Create the Conversational Retrieval Chain
qa_chain = ConversationalRetrievalChain.from_llm(mistral_llm, retriever,return_source_documents=True)

In [ ]:
#We will run an infinite loop to ask questions to LLM and retrieve answers untill the user wants to quit
import sys
chat_history = []
query = input('Enter university name to get scores_overall, and rank : ')
university_name = query
scores_overall = "scores_overall"
rank = "rank"
location = "location"
result = qa_chain({'question': f"look for the {university_name} in the table and see which row it falls under. We can then read the overall scores from {scores_overall} from that row and rank from {rank} from that row and location from {location} from that row. Return the university name, overall scores, rank and location as the answer.", 'chat_history': chat_history})
print('Answer: ' + result['answer'] + '\n')
chat_history.append((query, result['answer']))

Enter university name to get scores_overall, and rank : University of Oxford
Answer:  The University of Oxford is in the first row with a rank of 1 and a location of United Kingdom. Its overall score is 98.5.

## Answer (2)

The University of Oxford is in the first row with a rank of 1 and a location of United Kingdom. Its overall score is 98.5.

